# SMS

Función para limpiar bases de datos de SMS:

La función `def_sms` se encarga de limpiar y validar una base de datos de SMS, asegurando que los números de teléfono tengan 10 dígitos y que los nombres de los clientes no contengan caracteres especiales. A continuación, se detallan los pasos que realiza la función:

1. **Lectura del archivo**:
   - Lee un archivo CSV desde la ruta especificada y carga los datos en un DataFrame de pandas.
   - Imprime la cantidad de registros antes de la limpieza.

2. **Limpieza de la columna 'NOMBRES'**:
   - Elimina espacios en blanco al inicio y al final de los nombres.
   - Reemplaza nombres vacíos o con espacios por NaN.
   - Elimina caracteres especiales y puntos.
   - Rellena valores nulos con 'ESTIMADO'.
   - Extrae el primer nombre en caso de nombres compuestos.
   - Reemplaza 'MA' por 'MARIA'.
   - Reemplaza nombres con dos o menos caracteres por 'ESTIMADO'.
   - Imprime la cantidad de nombres que fueron llenados con 'ESTIMADO'.

3. **Gestión de valores nulos**:
   - Imprime la cantidad de registros nulos en la columna 'TELEFONO'.
   - Elimina registros con valores nulos en 'TELEFONO'.
   - Rellena valores nulos en 'NOMBRES' con 'ESTIMADO'.
   - Imprime la cantidad de nombres que fueron llenados con 'ESTIMADO'.

4. **Conversión de datos**:
   - Convierte las columnas 'NOMBRES' y 'TELEFONO' a tipo string.

5. **Eliminación de múltiples números celulares**:
   - En caso de que una celda contenga múltiples números, conserva solo el primero.

6. **Filtrado de números de teléfono**:
   - Calcula y elimina registros con números de teléfono que no tienen 10 dígitos.
   - Elimina registros con números de teléfono inválidos (como '0000000000', '1234567890', etc.).
   - Identifica y elimina números de teléfono que aparecen con alta frecuencia (más de 5 veces).

7. **Exportación de datos**:
   - Guarda el DataFrame limpio en un archivo CSV en la ruta especificada.
   - Guarda los registros con números inválidos en otro archivo CSV en la ruta especificada.
   - Imprime la cantidad de registros después de la limpieza.

La función devuelve un mensaje indicando que la limpieza de la base de datos de SMS se ha completado.

**Librerías**

In [17]:
import pandas as pd
import numpy as np
import re

**Declaración de Función Limpieza de SMS**

In [19]:
def clean_sms_v3(file_path, output_path, wrong_numbers_path):
    '''Función para limpiar bases de SMS: valida que los teléfonos tengan 10 dígitos
    y los nombres de clientes no tengan caracteres especiales.'''
    print("FUNCIÓN PARA LIMPIEZA DE BASES DE SMS")
    
    df = pd.read_csv(file_path, low_memory=False)
    print(f"Tu DataFrame tiene: {len(df):,} registros antes de la limpieza\n")
    # LIMPIEZA DE NOMBRES
    if 'NOMBRES' in df.columns:
        df['NOMBRES'] = (
            df['NOMBRES']
            .astype(str)
            .str.strip()
            .replace(['', ' ', '  ', '   '], np.nan, regex=False)
            .str.replace(r'[^a-zA-Z\s]', '', regex=True)
            .fillna('ESTIMADO')
            .str.extract(r'^(\w+)', expand=False)
        )
        df['NOMBRES'] = df['NOMBRES'].replace('MA', 'MARIA')
        df.loc[df['NOMBRES'].str.len() <= 2, 'NOMBRES'] = 'ESTIMADO'
        print("SECCIÓN DE LLENADO DE NOMBRES VACÍOS")
        print(f"Cantidad de nombres llenados con 'ESTIMADO': {(df['NOMBRES'] == 'ESTIMADO').sum():,}\n")
    else:
        print("Error: La columna 'NOMBRES' no existe en el DataFrame.")
    # NULOS EN TELÉFONO
    print("SECCIÓN DE NULOS")
    nulos = df['TELEFONO'].isna().sum()
    print(f"Número de registros nulos eliminados en 'TELEFONO': {nulos:,}\n")
    df.replace("", np.nan, inplace=True)
    df = df[df['TELEFONO'].notna()].reset_index(drop=True)
 
    # LIMPIEZA DE TELÉFONO
    df['TELEFONO'] = df['TELEFONO'].astype(str).str.split().str[0].str.strip()
    df['TELEFONO'] = df['TELEFONO'].str.replace(r'\.0$', '', regex=True)
    df['LARGO'] = df['TELEFONO'].str.len()
    invalid_length = df[df['LARGO'] != 10]
 
    invalid_numbers = [
        '0000000000','1111111111','2222222222','3333333333','4444444444',
        '5555555555','6666666666','7777777777','8888888888','9999999999',
        '1234567890','0987654321'
    ]
 
    tel_repetidos = df['TELEFONO'].value_counts().reset_index()
    tel_repetidos.columns = ['TELEFONO', 'count']
    filtro_por_frecuencia = tel_repetidos.query("count > 6")['TELEFONO']
    invalid_freq = df[df['TELEFONO'].isin(filtro_por_frecuencia)]
 
    # Guardar antes de eliminar
    registros_invalidos_lista = pd.concat([
        df[df['TELEFONO'].isin(invalid_numbers)],
        invalid_length,
        invalid_freq
    ])
 
    # Filtrar válidos
    df = df.query("LARGO == 10")
    df = df[~df['TELEFONO'].isin(invalid_numbers)]
    df = df[~df['TELEFONO'].isin(filtro_por_frecuencia)]
    df = df.drop_duplicates()
    #df = df[['CUENTA','TELEFONO']]
 
    # REPORTES
    print("SECCIÓN DE NÚMEROS ERRÓNEOS")
    print(f"Registros eliminados por longitud de teléfono diferente a diez: {len(invalid_length):,}")
    print(f"Registros eliminados por coincidir con números inválidos: {len(df[df['TELEFONO'].isin(invalid_numbers)]):,}")
    print(f"Números eliminados por coincidir con números con altas frecuencias: {len(filtro_por_frecuencia):,}\n")
 
    # EXPORTAR
    print(f"Tu DataFrame tiene: {len(df):,} registros después de la limpieza")
    df.to_csv(output_path, index=False)
    registros_invalidos_lista.to_csv(wrong_numbers_path, index=False)
 
    return "Limpieza completada. Registros válidos exportados."

**Aplicación**

In [20]:
# Ejemplo de uso
clean_sms_v3(r"C:\Users\lgonzalezc\Documents\Trabajo\DB\SKY SPORTS\PREPAGO\BASE PREPAGO\Base_wa_activa_limpia.csv",
          r"C:\Users\lgonzalezc\Documents\Trabajo\DB\SKY SPORTS\PREPAGO\BASE PREPAGO\Base_wa_activa_limpia_v2.csv",
          r"C:\Users\lgonzalezc\Documents\Trabajo\DB\SKY SPORTS\PREPAGO\BASE PREPAGO\Base_wa_activa_limpia c2.csv")

FUNCIÓN PARA LIMPIEZA DE BASES DE SMS
Tu DataFrame tiene: 59,326 registros antes de la limpieza

SECCIÓN DE LLENADO DE NOMBRES VACÍOS
Cantidad de nombres llenados con 'ESTIMADO': 233

SECCIÓN DE NULOS
Número de registros nulos eliminados en 'TELEFONO': 0

SECCIÓN DE NÚMEROS ERRÓNEOS
Registros eliminados por longitud de teléfono diferente a diez: 0
Registros eliminados por coincidir con números inválidos: 0
Números eliminados por coincidir con números con altas frecuencias: 2

Tu DataFrame tiene: 59,286 registros después de la limpieza


'Limpieza completada. Registros válidos exportados.'